# scaredy plot is an offshoot of scaredy rat
### Generates plots of velocity over time

#### To run, change variables in specified cell, then go to Kernel > Restart & Run All
Known possible issues:
 - for whatever reason, the raw data column for Tone 10 doesn't have a space (i.e. "Tone10" instead of "Tone 10). fix this by ctrl-F-ing the raw data and making that replacement. it will throw an error if left unchanged.
 - there won't be data for the 12th tone due to indexing issues (sometimes recording ends before completing time bins). fix tbd. for now, it just ends at Tone 11.
 - various things need to be changed if there is a different number of tones - email.
 - in the **animal_read** function, the indexes are specific to the ethovision output. sometimes there are other rows in the header, so the numbers will have to be changed. if that function throws an error, this may be the issue. for reference, the animal ID is row 33 on the excel file, so the corresponding index is **df.iloc[31]**. if the animal ID were on row 34, you would change the index to **df.iloc[32]**.
 - in the same vein, the header row will need to be readjusted: if header row is on row 36 in the data file, then **skiprows = 35**.

for others, email me at annajxli@gmail.com

In [ ]:
%matplotlib inline

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import math
import time

In [ ]:
def animal_read(filename,sheet):
    '''
    Returns corresponding animals for sheets
    Reads data into dataframe with long header removed.
    Uses correct labels as column labels.
    Uses time as row indexes.
    '''
    filepath = os.path.join(inpath,filename)
    df = pd.read_excel(filepath,sheetname=sheet,skip_footer=34,index_col=0)
    animal = df.iloc[32][0]
    print(filename + " " + sheet + " is " + animal)
    
    df = pd.read_excel(filepath,sheetname=sheet,skiprows=35,index_col=0,headers=0)
    df = df[1:]
    df.replace(to_replace='-',value=0,inplace=True)
    return(animal,df)

In [ ]:
def find_tone_vels(df,i):
    '''
    '''
    tone = pd.DataFrame()
    num = str(i)
    label = 'Tone ' + num
    tone = pd.DataFrame(df[df[label] == 1])
    
    tone = tone['Velocity']
    return(tone)

In [ ]:
def find_shock_vels(df, i):
    '''
    '''
    sresponse = pd.DataFrame()
    num = str(i)
    label = 'Tone ' + num
    
    tone = df[df[label] == 1] #Tone dataframe
    toneend = tone.iloc[-1] #Time for tone end
    
    starttime = math.floor(toneend['Recording time']) #Time for sresponse start
    endtime = math.floor(toneend['Recording time'] + 5) #Time for sresponse end
    itime = df.index.get_loc(starttime,method='bfill') #Index for sresponse start
    etime = df.index.get_loc(endtime,method='bfill') #Index for sresponse end
    
    sresponse = df.iloc[itime:etime] #df for sresponse
    sresponse = sresponse['Velocity']
    
    return(sresponse)

# Make changes here for input/output folders and file/sheet names.

In [ ]:
## This is the folder with the raw data
## Be sure to put r before the path
## Easiest to go to correct folder, then copy/paste its path
inpath = r'C:\Users\Anna\Documents\Shansky Lab\170802 ethovision analysis\raw data'

## This is where the output goes
## Make sure the folder exists already.
## The file does not need to exist; it will be created.
outpath = r'C:\Users\Anna\Documents\Shansky Lab\170802 ethovision analysis\outputs'

filename = 'Raw data-BRF Summer July 2017-Trial     7.xlsx' ## include .xlsx
sheetname = 'Track-Arena 1-Subject 1'

## Function calls to read animal data
#### These can take a while to run due to the size of the files.

In [ ]:
ID,anim = animal_read(filename,sheetname)
vels = pd.DataFrame(anim['Velocity'])

#tones = find_tones(anim)
#shocks = find_shock_responses(anim)

In [ ]:
plt.style.use('seaborn-white')
plt.figure(figsize=(16,8),facecolor='white',edgecolor='white')
# Plots main velocity
plt.plot(vels,color='k',linewidth=0.1)

# Loops through tones, plots each one.
i = 1
while i <= 12:
    tone = find_tone_vels(anim,i)
    plt.plot(tone,color='c',linewidth=1)
    i += 1

# Loops through shocks, plots each one.
i = 6
while i <= 11:
    sresponse = find_shock_vels(anim,i)
    plt.plot(sresponse,color='m',linewidth=1)
    i += 1

sns.despine(left=True, bottom=True, right=True)
plt.title(ID)
plt.ylabel('Velocity (cm/s)')
plt.xlabel('Trial time (s)')

fname = ID + '-vel-plot'
plt.savefig(fname, dpi=300)

plt.show()
plt.close()